In [ ]:
from google.colab import drive # 학습 중 나오는 ckpt 저장 위한 경로 설정
drive.mount('/content/drive')

In [2]:
!pip install git+https://github.com/huggingface/datasets.git@master # huggingface 데이터 다운로드

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git (to revision master) to /tmp/pip-req-build-kbhoo2t3
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-kbhoo2t3
     |████████████████████████████████| 212 kB 4.1 MB/s 
     |████████████████████████████████| 140 kB 56.1 MB/s 
     |████████████████████████████████| 1.1 MB 3.8 MB/s 
     |████████████████████████████████| 86 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 30.6 MB/s 
     |████████████████████████████████| 127 kB 63.5 MB/s 
     |████████████████████████████████| 271 kB 59.4 MB/s 
     |████████████████████████████████| 144 kB 15.9 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Created wheel for datasets: filename=datasets-2.2.3.dev0-py3-none-any.whl size=354866 sha256=47a43a4ce038fed9301afd9269c6e569cfd469e1e70fcdb4a4b2eb0ca9b40481
  Stored in dire

In [3]:
!pip install git+https://github.com/hyunwoongko/kobart-transformers.git # kobart transformer 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hyunwoongko/kobart-transformers.git to /tmp/pip-req-build-wugtzo31
  Running command git clone -q https://github.com/hyunwoongko/kobart-transformers.git /tmp/pip-req-build-wugtzo31
  Created wheel for kobart-transformers: filename=kobart_transformers-0.1.4-py3-none-any.whl size=3368 sha256=2c83155fdf17659c97636c503457309c126266b49cc160877924c57d87ae8618
  Stored in directory: /tmp/pip-ephem-wheel-cache-yzeq2am9/wheels/f1/aa/a4/10278d9a320156ab15659d444fdbe6243adb3c50796cba1644
Successfully built kobart-transformers


In [4]:
!pip install transformers # transformer 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.0 MB/s 


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [6]:
import torch
import pandas as pd
from kobart_transformers import get_kobart_tokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MBartModel, MBartConfig ,PretrainedConfig
from transformers import MBartTokenizer, TFMBartModel , TFTrainer , TFTrainingArguments
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

In [7]:
!gdown 1Ozs0QptMohSwNdwa_Vg5yqO8_TZuTsv1

Downloading...
From: https://drive.google.com/uc?id=1Ozs0QptMohSwNdwa_Vg5yqO8_TZuTsv1
To: /content/last.csv
100% 31.2M/31.2M [00:00<00:00, 52.0MB/s]


In [8]:
df = pd.read_csv('/content/last.csv')

In [9]:
print(df.isnull().sum())

ko    0
en    0
dtype: int64


In [10]:
df = df.dropna()

In [11]:
print(df.isnull().sum())

ko    0
en    0
dtype: int64


In [12]:
len(df) # 결측치를 제외한 데이터 개수

126185

In [13]:
ko = df['ko']
en = df['en']

In [14]:
ko.to_csv('ko.csv',index=False)
en.to_csv('en.csv',index=False)

In [15]:
data = []
with open("/content/en.csv") as f1, open("/content/ko.csv") as f2:
    for src, tgt in zip(f1, f2):
      data.append(
          {
                  "ko": tgt.strip(),
                  "en": src.strip()
          }
      )
print(f'total size of data is {len(data)}')

total size of data is 126186


In [16]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = pd.DataFrame(data)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [17]:
hg_dataset

Dataset({
    features: ['ko', 'en'],
    num_rows: 126186
})

In [18]:
type(hg_dataset)

datasets.arrow_dataset.Dataset

In [19]:
data = hg_dataset.train_test_split(test_size=0.2)

In [21]:
print(data['train'])
print(data['test'])

Dataset({
    features: ['ko', 'en'],
    num_rows: 100948
})
Dataset({
    features: ['ko', 'en'],
    num_rows: 25238
})


In [22]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ko_KR")

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["en"]] # 영어원문을 input으로 선언
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) 
    # tokenizer 작업한 영어를 model_input으로 선언

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["ko"], max_length=1024, truncation=True)
        # tokenizer 작업한 한글을 labels로 선언

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [24]:
tokenized_data = data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f7b97f35170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/101 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [25]:
tokenized_data  # ko : 한글 논문
                # en : 영어 논문
                # input_ids :  integer 상태의 영어
                # labels :  integer 상태의 한글
                # attention_mask :  masking된 attention

DatasetDict({
    train: Dataset({
        features: ['ko', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100948
    })
    test: Dataset({
        features: ['ko', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25238
    })
})

In [26]:
tokenized_data['test'][1]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'en': '"According to this configuration, the security element 172 may transmit a response signal to a command requesting a security related service."',
 'input_ids': [44,
  39450,
  32271,
  214,
  47,
  903,
  180346,
  4,
  70,
  48031,
  12830,
  92111,
  1543,
  71872,
  10,
  57553,
  26073,
  47,
  10,
  75101,
  50336,
  214,
  10,
  48031,
  62548,
  4516,
  1242,
  2,
  250004],
 'ko': '"일실시예에 따르면, 보안 엘리먼트(172)는 보안 관련 서비스를 요청하는 명령에 대한 응답 신호를 전송할 수 있다."',
 'labels': [44,
  1599,
  6292,
  2166,
  17320,
  480,
  81245,
  4,
  135108,
  6,
  71939,
  2085,
  206393,
  132,
  130252,
  16,
  769,
  135108,
  20448,
  88160,
  96045,
  2123,
  213243,
  480,
  6804,
  227127,
  6521,
  7346,
  688,
  199673,
  2461,
  1020,
  3162,
  1242,
  2,
  250014]}

In [ ]:
type(tokenized_data)

datasets.dataset_dict.DatasetDict

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    save_strategy='epoch',
    

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: ko, en. If ko, en are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100948
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 189285


Epoch,Training Loss,Validation Loss
1,0.843600,0.789460
2,0.684200,0.722583
3,0.546200,0.717286
4,0.431400,0.740622
5,0.335800,0.791957
6,0.272400,0.853283
7,0.197200,0.931563
8,0.143800,0.993216
9,0.111800,1.042014
10,0.083300,1.086850


The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: ko, en. If ko, en are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25238
  Batch size = 8
Saving model checkpoint to /content/checkpoint-12619
Configuration saved in /content/checkpoint-12619/config.json
Model weights saved in /content/checkpoint-12619/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-12619/tokenizer_config.json
Special tokens file saved in /content/checkpoint-12619/special_tokens_map.json
added tokens file saved in /content/checkpoint-12619/added_tokens.json
The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: ko, en. If ko, en are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this mess

Epoch,Training Loss,Validation Loss
1,0.843600,0.789460
2,0.684200,0.722583
3,0.546200,0.717286
4,0.431400,0.740622
5,0.335800,0.791957
6,0.272400,0.853283
7,0.197200,0.931563
8,0.143800,0.993216
9,0.111800,1.042014
10,0.083300,1.086850


KeyboardInterrupt: ignored

In [ ]:
tokenizer.save_pretrained('/content/pretrain_token')

tokenizer config file saved in /content/pretrain_token/tokenizer_config.json
Special tokens file saved in /content/pretrain_token/special_tokens_map.json
added tokens file saved in /content/pretrain_token/added_tokens.json


('/content/pretrain_token/tokenizer_config.json',
 '/content/pretrain_token/special_tokens_map.json',
 '/content/pretrain_token/sentencepiece.bpe.model',
 '/content/pretrain_token/added_tokens.json')

번역기

In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer


model = MBartForConditionalGeneration.from_pretrained("/content/checkpoint-113571")
tokenizer = tokenizer.from_pretrained('/content/checkpoint-113571')
article = "Build a Machine Learning web application from scratch in Python with Streamlit. We use real world data to build a machine learning model. In the first part you learn how we analyze the data and build our model, and in the second part we build the web app using streamlit."
inputs = tokenizer(article, return_tensors="pt")
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
translate = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
print(translate)

loading configuration file /content/checkpoint-113571/config.json
Model config MBartConfig {
  "_name_or_path": "facebook/mbart-large-50",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "

['Streamlit으로 Python에서부터 머신러닝 웹 애플리케이션을 구체화 시켜 놓는다. 실세계 데이터를 이용해 머신러닝 모델을 구축한다. 첫 번째 파트에서는 데이터 분석 방법을 배우고 머신러닝 모델을 구축하는 두 번째 파트에서는  streamlit을 이용해 머신러닝을 이용한 웹 앱을 구축한다.']


In [ ]:
print(translate)

['Streamlit으로 Python에서부터 머신러닝 웹 애플리케이션을 구체화 시켜 놓는다. 실세계 데이터를 이용해 머신러닝 모델을 구축한다. 첫 번째 파트에서는 데이터 분석 방법을 배우고 머신러닝 모델을 구축하는 두 번째 파트에서는  streamlit을 이용해 머신러닝을 이용한 웹 앱을 구축한다.']
